In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses. 

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [3]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
PROJECT_ID = "qwiklabs-gcp-01-177d3e4db61c"  # @param {type:"string"}
LOCATION = "us-east4"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [5]:
model = GenerativeModel("gemini-1.5-flash")

In [6]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [7]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some name ideas for a flower shop specializing in dried flowers, playing on different themes:

**Whimsical & Romantic:**

* Whispered Petals
* Everlasting Blooms
* Dried Dreams
* The Paper Garden
* Bloom & Wither
* The Dusty Rose
* Timeless Bouquet
* The Dried Flower Cottage
* The Bloom & Botanical

**Modern & Minimalist:**

* Wild & Dried
* The Dried Bloom Co.
* Bloom & Preserve
* The Botanist's Shelf
* The Flower Archive
* The Dried Bouquet
* Simply Dried
* Gather & Bloom

**Creative & Unique:**

* The Herbarium
* The Flower Alchemist
* The Sun-Kissed Bouquet
* The Paper Flower Shop
* The Dried Flower Bar
* The Floral Canvas
* Bloom & Hues
* The Vintage Bloom
* The Botanical Muse

**Tips for Choosing a Name:**

* **Keep it short and memorable.**
* **Consider your target audience.**
* **Make it relevant to your brand identity.**
* **Check for availability.**
* **Get feedback from others.**

Ultimately, the best name for your flower shop is one that you love and that accuratel

✅ Recommended. The prompt below is to the point and concise.

In [8]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Names for a dried flower shop:

**Short & Sweet:**

* Bloom & Preserve
* Everlasting Blooms
* Dried & Delightful
* The Dried Flower Co.
* Petal & Stem
* Willow & Wildflower
* Botanicals & Blooms

**Unique & Evocative:**

* The Paper Garden
* Whispers of Wild
* Sun-Kissed Blooms
* Timeless Blooms
* The Petal Archive
* Dried Dreams
* The Botanical Bar

**Descriptive & Playful:**

* Forever in Bloom
* The Flower Alchemist
* Wildflower Sanctuary
* The Dried Bouquet Bar
* Bloom & Dust
* The Floral Muse
* The Dusty Rose

**Bonus:**

* Use your own name/initials for a personal touch.
* Consider incorporating your location or a local landmark in the name.
* Use a play on words related to dried flowers (e.g., "Timeless Blooms," "Forever in Bloom").

**Tip:** Check for available domain names and social media handles before settling on a name. 



### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [9]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known place in the universe where life is known to exist. It's a dynamic and diverse planet with a fascinating history, complex systems, and a vibrant biosphere. 

**Here's a glimpse of Earth:**

**Physical Features:**

* **Size:**  Earth has a diameter of about 12,742 kilometers (7,926 miles).
* **Shape:** It's not perfectly spherical but slightly flattened at the poles and bulging at the equator, an oblate spheroid.
* **Composition:** The planet is composed of a solid inner core, a liquid outer core, a mantle, and a thin crust. 
* **Atmosphere:** Earth has a protective atmosphere composed mainly of nitrogen (78%) and oxygen (21%). 
* **Hydrosphere:** About 71% of Earth's surface is covered in water, forming oceans, lakes, rivers, and ice.
* **Landmasses:** The remaining 29% of Earth's surface is made up of continents and islands.

**Life on Earth:**

* **Biosphere:**  Earth is home to an incredible diversi

✅ Recommended. The prompt below is specific and well-defined.

In [10]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What Makes Earth Unique:

**1. Liquid Water:** Earth is the only known planet in our solar system with abundant liquid water on its surface. This is crucial for life as we know it.

**2. Plate Tectonics:**  Earth's crust is divided into tectonic plates that constantly shift and interact, causing earthquakes, volcanoes, and mountain formation. This dynamic process plays a vital role in regulating the planet's climate and creating diverse environments.

**3. Atmosphere:**  Earth's atmosphere is a protective shield, regulating temperature, filtering harmful radiation, and allowing for weather patterns. Its composition, including the presence of oxygen, is unique and supports life.

**4. Magnetic Field:** Earth's molten iron core generates a strong magnetic field, acting like a shield against harmful solar wind and radiation. 

**5. Diverse Life:** Earth is teeming with a vast array of life forms, from microscopic bacteria to towering trees and intelligent humans. This incredible biodiv

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [11]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Here's a breakdown of boiling water and the blue sky:

**Boiling Water**

* **Method:** The best method depends on your needs and resources. Here are a few common options:
    * **Stovetop:**  Simple, efficient, and works for most quantities of water. Use a pot with a flat bottom for even heating. 
    * **Electric Kettle:** Fast, convenient, and energy-efficient for smaller amounts of water.
    * **Microwave:**  Quick for small quantities, but be cautious of uneven heating and potential for splattering.
* **Why it Works:**  When you heat water, the molecules move faster and faster.  Eventually, they gain enough energy to break free from the liquid state and become steam (water vapor).  The boiling point of water is 100°C (212°F) at standard atmospheric pressure.

**Why the Sky is Blue**

* **Scattering:**  Sunlight is a mix of all colors (think of a rainbow).  As sunlight enters the Earth's atmosphere, it interacts with tiny particles like nitrogen and oxygen molecules. This interact

✅ Recommended. The prompts below asks one task a time.

In [12]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

There's no single "best" method, as it depends on your needs and resources. Here's a breakdown of different methods and their pros and cons:

**1. Stovetop:**

* **Pros:** Most common, versatile, can boil large quantities, good for making tea or coffee.
* **Cons:** Takes longer to heat up, can be inefficient if using an old stove, risk of burning if left unattended.

**2. Electric Kettle:**

* **Pros:** Fastest method, energy-efficient, convenient, often has temperature control for different drinks.
* **Cons:** Limited capacity, can be noisy, less versatile than stovetop.

**3. Microwave:**

* **Pros:** Very fast, convenient for small quantities, good for instant beverages.
* **Cons:** Doesn't always boil water evenly, can be less safe if used improperly, may not be suitable for all containers.

**4. Camping Stove (Gas or Propane):**

* **Pros:** Portable and ideal for outdoor use, efficient for camping or emergencies.
* **Cons:** Requires fuel source, may be less convenient for everyd

In [13]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky is blue because of a phenomenon called **Rayleigh scattering**. Here's how it works:

* **Sunlight is made up of all colors of the rainbow.** This is called the visible spectrum.
* **The Earth's atmosphere is filled with tiny particles, mostly nitrogen and oxygen molecules.**
* **When sunlight enters the atmosphere, these particles scatter the light in all directions.** However, they scatter shorter wavelengths (like blue and violet) much more effectively than longer wavelengths (like red and orange).
* **This preferential scattering of blue light is why we see a blue sky.** Our eyes are more sensitive to blue than violet, so we perceive the sky as blue rather than violet.

**Here's a simple analogy:** Imagine a box filled with tiny marbles. If you shine a white light on the box, the marbles will scatter the light in all directions. But smaller marbles will scatter shorter wavelengths of light more effectively than larger marbles. So, if you look at the box from the side, you'l

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [14]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To find out what day it is today, please check a calendar or your device's clock. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [15]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan is a city full of amazing sights! To give you the best recommendation, I need a little more information. What are you most interested in seeing?  

* **Art and history?** Then you might love the Duomo, Leonardo da Vinci's Last Supper, or the Sforza Castle. 
* **Fashion and shopping?**  The Quadrilatero della Moda is a must-see. 
* **Something unique and off the beaten path?**  Maybe you'd enjoy the Navigli canals or the Brera district. 

Tell me more about your interests, and I can help you narrow down the perfect place for sightseeing in Milan! 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [16]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help with travel planning, not to give dinner recommendations. 

Would you like me to help you find some great restaurants in Milan?  I can help you search for Italian cuisine,  local specialties, or even something more international! 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [17]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

## Programming Activities for High School Students:

**Level 1: Beginner Friendly:**

* **Build a simple text-based game:**  Start with a classic like "Guess the Number" or "Hangman". This helps you learn basic input/output, loops, and conditional statements.
* **Create a basic calculator:**  Practice arithmetic operations and user input. This also allows you to explore different data types.
* **Develop a simple website:**  Learn HTML, CSS, and JavaScript to create a basic webpage with interactive elements.  

**Level 2: Intermediate Challenges:**

* **Build a simple data visualization tool:**  Use Python libraries like matplotlib or Plotly to create charts and graphs. You can explore data sets from your interests, like sports stats or video game scores.
* **Create a chatbot:**  Learn about natural language processing and design a simple chatbot that can respond to basic questions.
* **Develop a command-line tool:**  This helps you understand how to interact with the operating system a

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [18]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

It's great that you're exploring programming languages! All three languages you mentioned have their strengths and weaknesses. Here's a breakdown to help you decide:

**a) Python:**

* **Pros:**  
    * **Beginner-friendly:** Python's syntax is simple and readable, making it excellent for beginners.
    * **Versatility:** Widely used in web development, data science, machine learning, scripting, and more.
    * **Large community:**  Extensive resources, tutorials, and support available online.
    * **High demand:**  Python skills are highly sought after in various industries. 

* **Cons:**  
    * **Not the best for web front-end:** While Python can be used for web development, it's not as popular for front-end work as JavaScript.

**b) JavaScript:**

* **Pros:**  
    * **Dominates web development:** Essential for interactive websites, web applications, and front-end development.
    * **Growing ecosystem:** Used in game development, mobile apps, and server-side programming (Node.js)

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [19]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [20]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [21]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **positive** 

Here's why:

* **"Something surprised me about this video - it was actually original."**  This implies a positive surprise and appreciation for something different.
* **"It was not the same old recycled stuff that I always see."**  This further highlights the positive aspect of the video being original and not repetitive.
* **"Watch it - you will not regret it."** This is a strong recommendation and indicates a positive opinion. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.